In [2]:
import os
import importlib
import re, array, csv, datetime, glob, json, math, random, stat
import pytz, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# This is a utility function for running other ipython notebooks
def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


# Load utility functions from another ipython notebook
root_dir = "/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/"
os.chdir(root_dir + "linRegModel/pardump_example/")
exec_ipynb("pardumpdump-randy-amy-util.ipynb")
os.chdir(root_dir + "linRegModel")
exec_ipynb("./cachedHysplitRunLib.ipynb")
exec_ipynb("../../src/python-utils/utils.ipynb")
os.chdir(root_dir + "automate-plume-viz/")

In [6]:
# Specify the starting and ending date, also the time offset
start = "2019-01-01"
end = "2020-01-01"
offset = pd.Timedelta(3, unit="h")

# Create rows in the EarthTime document
start_d = pd.date_range(start=start, end=end, closed="left", tz="US/Eastern") - offset
end_d = pd.date_range(start=start, end=end, closed="right", tz="US/Eastern") - pd.Timedelta(10, unit="m") - offset
df_template = pd.read_csv("data/earth_time_template.csv")
df = df_repeated = pd.concat([df_template]*len(start_d), ignore_index=True)
file_name = "plume_" + end_d.strftime("%Y%m%d")
df["Start date"] = start_d.tz_convert("UTC").strftime("%Y%m%d%H%M%S")
df["End date"] = end_d.tz_convert("UTC").strftime("%Y%m%d%H%M%S")
df["Share link identifier"] = file_name
df["Name"] = "PARDUMP " + end_d.strftime("%Y-%m-%d")
df["URL"] = "https://cocalc-www.createlab.org/pardumps/" + file_name + ".bin"

# Save rows of EarthTime CSV layers to a file so that we can copy and paste
p = "data/earth_time.csv"
df.to_csv(p, index=False)
os.chmod(p, 0o777)

# Create rows of share URLs
et_root_url = "https://davos2019.earthtime.org/explore#"
v = "v=40.43266,-79.93955,9.318,latLng&"
t = "t=3.83&"
ps = "ps=50&"
l = "l=bdrk,plume_20190101,smell_my_city_pgh_reports&"
sd = "startDwell=0&"
ed = "endDwell=0"
df_url = []
for index, row in df.iterrows():
    bt = "bt=" + row["Start date"] + "&"
    et = "et=" + row["End date"] + "&"
    full_url = et_root_url + v + t + ps + l + bt + et + sd + ed
    df_url.append(full_url)
df_url = pd.DataFrame(data=df_url)

# Save rows of EarthTime urls to a file
p = "data/earth_time_urls.csv"
df_url.to_csv(p, index=False)
os.chmod(p, 0o777)

'https://davos2019.earthtime.org/explore#v=40.43266,-79.93955,9.318,latLng&t=3.83&ps=50&l=bdrk,plume_20190101,smell_my_city_pgh_reports&bt=20190101020000&et=20190102015000&startDwell=0&endDwell=0'

In [7]:
# Location of the sources of pollution
sources = [
    DispersionSource(name='Irvin',lat=40.328015, lon=-79.903551, minHeight=0, maxHeight=50),
    DispersionSource(name='ET',lat=40.392967, lon=-79.855709, minHeight=0, maxHeight=50),
    DispersionSource(name='Clairton',lat=40.305062, lon=-79.876692, minHeight=0, maxHeight=50),
    DispersionSource(name='Cheswick',lat=40.538261, lon=-79.790391, minHeight=0, maxHeight=50),
]


# Run the simulation
#   start_time_eastern: for different dates, use format "2020-03-30 00:00"
#   o_file: file name to save the simulation result
#   sources: location of the sources of pollution
#   emit_time_hrs:  affects the emission time for running each Hysplit model
#   duration: total time (in hours) for the simulation, use 24 for a total day, use 12 for testing
def simulate(start_time_eastern, o_file, sources=sources, emit_time_hrs=1, duration=24):
    print("="*100)
    print("="*100)
    print("start_time_eastern: %s" % start_time_eastern)
    print("o_file: %s" % o_file)
    # Run simulation and get the folder list (the generated files are cached)
    path_list = []
    for source in sources:
        path_list += getMultiHourDispersionRunsParallel(
                source,
                parse_eastern(start_time_eastern),
                emit_time_hrs,
                duration,
                HysplitModelSettings(initdModelType=InitdModelType.ParticleHV, hourlyPardump=False))
    print(path_list)
    # Save pdump text files (the generated files are cached)
    pdump_txt_list = []
    for folder in path_list:
        if not findInFolder(folder,'*.gz') and not findInFolder(folder,'PARDUMP*.txt'):
            pdump = findInFolder(folder,'PARDUMP.*')
            print(pdump)
            cmd = "/opt/hysplit/exec/par2asc -i%s -o%s" % (pdump, pdump+".txt")
            #cmd = f"/opt/hysplit/exec/par2asc -i{pdump} -o{pdump}.txt"
            if pdump.find('.txt') == -1:
                pdump_txt_list.append(pdump+".txt")
            subprocess_check(cmd)
        if findInFolder(folder,'PARDUMP*.txt'):
            pdump_txt = findInFolder(folder,'PARDUMP*.txt')
            pdump_txt_list.append(pdump_txt)
    print(pdump_txt_list)
    # Add color
    cmap = "viridis"
    c = plt.get_cmap(cmap)
    c.colors
    colors = np.array(c.colors)
    colors *= 255
    colormap = np.uint8(colors.round())
    colormap = colormap.reshape([1,256,3])
    cmaps = [
        [[250, 255, 99]],
        [[250, 255, 99],[99, 255, 206]],
        [[250, 255, 99],[99, 255, 206],[206, 92, 247]],
        [[250, 255, 99],[99, 255, 206],[206, 92, 247],[255, 119, 0]]
    ]
    print("Creating %s" % o_file)
    points = create_multisource_bin(pdump_txt_list, o_file, len(sources), False, cmaps, duration)
    print("Created %s" % o_file)
    os.chmod(o_file, 0o777)


# Prepare the list of dates for running the simulation
start_time_eastern_all = start_d.strftime("%Y-%m-%d %H:%M").values
o_root = "/projects/cocalc-www.createlab.org/pardumps/"
o_file_all = o_root + file_name.values + ".bin"

# For each date, run the simulation
#for i in range(len(start_time_eastern_all)):
for i in [91,112]: # for testing purposes
    if os.path.isfile(o_file_all[i]): # skip if the file exists
        print("File already exists %s" % o_file_all[i])
        continue
    simulate(start_time_eastern_all[i], o_file_all[i])

start_time_eastern: 2019-04-01 21:00
o_file: /projects/cocalc-www.createlab.org/pardumps/plume_20190402.bin
SimpleThreadPoolExecutor succeeded: all 24 jobs completed
SimpleThreadPoolExecutor succeeded: all 24 jobs completed
SimpleThreadPoolExecutor succeeded: all 24 jobs completed
SimpleThreadPoolExecutor succeeded: all 24 jobs completed
['/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190401_210000-0400_1h_24h_0', '/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190401_220000-0400_1h_23h_0', '/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190401_230000-0400_1h_22h_0', '/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190402_000000-0400_1h_21h_0', '/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.3280

/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190401_220000-0400_1h_23h_0/PARDUMP.h22.txt


/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190401_230000-0400_1h_22h_0/PARDUMP.h23.txt


/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190402_000000-0400_1h_21h_0/PARDUMP.h0.txt


/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.328015,-79.903551_0-50/20190402_010000-0400_1h_20h_0/PARDUMP.h1.txt


...

/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.305062,-79.876692_0-50/20190423_110000-0400_1h_10h_0/PARDUMP.h11.txt


/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.305062,-79.876692_0-50/20190423_110000-0400_1h_10h_0/PARDUMP.h11.txt


/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.305062,-79.876692_0-50/20190423_120000-0400_1h_9h_0/PARDUMP.h12.txt


/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.305062,-79.876692_0-50/20190423_120000-0400_1h_9h_0/PARDUMP.h12.txt


/projects/9ab71616-fcde-4524-bf8f-7953c669ebbb/air-src/linRegModel/dispersionCache/40.305062,-79.876692_0-50/20190423_130000-0400_1h_8h_0/PARDUMP.h13.txt
